# MovieLens Dataset

Import all packages to extract data from csv file 'ratings.csv' and store it in a dataframe.

In [1]:
import numpy as np
import pandas as pd
import pickle             # store attributes in pkl files
import collections        # convert list into hash-table (values denoting key frequency in list)

In [2]:
# File excel with entries separated by column 
# First row displays the header of the DataFrame

df = pd.read_csv('ml-latest-small/ratings.csv',sep = ',')

According to MovieLens documentation:

This dataset describes 5-star rating from MovieLens. It contains 100004 ratings and 1296 tag applications across 9125 movies. These data were created by 671 users between January 09, 1995 and October 16, 2016.

### Summary Statistics

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
# There are fewer movies than the number reported by the documentation
print('ratings:',len(df))
df.nunique()

ratings: 100004


userId         671
movieId       9066
rating          10
timestamp    78141
dtype: int64

In [5]:
# The ratings range from 0.5 t0 5
df.rating.sort_values().unique()

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [6]:
# No Nan values
print(df.isna().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [7]:
# Each user didn't watch the same movie twice or more
len(df.groupby(['userId','movieId']))

100004

In [8]:
# Convert timestamp do datetime and drop timestamp column
df['date'] = pd.to_datetime(df['timestamp'],unit='s').dt.date
df.drop('timestamp',axis=1,inplace=True)

In [9]:
# start and end date are as described in the documentation
print(df.date.min())
print(df.date.max())

1995-01-09
2016-10-16


### Movies Dataframe

In [10]:
df_movie = pd.read_csv('ml-latest-small/movies.csv',sep = ',')

In [11]:
df_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
# Extract year from title attribute
df_movie['year'] = df_movie.title.str.extract("\((\d{4})\)", expand=True)

In [13]:
df_movie.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


#### No genre

In [14]:
# Some titles do not have a genre
df_movie[df_movie.genres.str.contains('no genres')].head()

,movieId,title,genres,year
7724,83829,Scorpio Rising (1964),(no genres listed),1964
8732,117192,Doctor Who: The Time of the Doctor (2013),(no genres listed),2013
8784,122888,Ben-hur (2016),(no genres listed),2016
8799,126106,Beastie Boys: Sabotage (1994),(no genres listed),1994
8829,128616,As We Were Dreaming (2015),(no genres listed),2015


In [15]:
idx = df_movie[df_movie.genres.str.contains('no genres')].index.values

In [16]:
# Add genres manually
genre_added = ['Musical','Drama|Fantasy|Adventure','Drama','Musical','Drama','Crime|Thriller','Comedy','Comedy|Drama','Action|Drama','Documentary','Comedy|Fantasy','Drama|Thriller','Drama','Thriller','Drama|War','Drama',
'Documentary|Thriller','Documentary']

In [17]:
df_movie.loc[idx,'genres'] = genre_added

#### No year

In [18]:
# Some titles do not display year or it is encoded in different format
df_movie[df_movie.year.isnull()]

,movieId,title,genres,year
8505,108548,"Big Bang Theory, The (2007-)",Comedy,NaN
8507,108583,Fawlty Towers (1975-1979),Comedy,NaN
9017,143410,Hyena Road,Drama|War,NaN
9063,151307,The Lovers and the Despot,Documentary|Thriller,NaN
9118,162376,Stranger Things,Drama,NaN
9124,164979,"Women of '69, Unboxed",Documentary,NaN


In [19]:
idx2 = df_movie[df_movie.year.isnull()].index.values

In [20]:
# Add year manually
df_movie.loc[idx2,'year'] = [2007,1975,2015,2016,2016,2014]

In [21]:
df_movie.year = pd.to_numeric(df_movie.year)

#### Repetition of titles - Hamlet and War of the Worlds 

In [22]:
df_movie.title.value_counts().head()

War of the Worlds (2005)           2
Hamlet (2000)                      2
War Horse (2011)                   1
They All Laughed (1981)            1
High and the Mighty, The (1954)    1
Name: title, dtype: int64

In [23]:
# Hamlet (2000) repeated twice
df_movie[df_movie.title.str.contains('Hamlet')]

,movieId,title,genres,year
1146,1411,Hamlet (1996),Crime|Drama|Romance,1996
1503,1941,Hamlet (1948),Drama,1948
2260,2820,Hamlet (1964),Drama,1964
2872,3598,Hamlet (2000),Crime|Drama|Romance|Thriller,2000
2975,3723,Hamlet (1990),Drama,1990
7151,65665,Hamlet (2000),Drama,2000


In [24]:
# Drop duplicate from movie dataframe
df_movie.drop(7151,inplace=True)

In [25]:
df[df.movieId == 3598]

,userId,movieId,rating,date
1668,15,3598,3.0,2001-08-16
2881,17,3598,4.0,2005-09-23
19345,129,3598,4.0,2000-08-03
31174,222,3598,5.0,2000-06-13
56558,407,3598,5.0,2000-07-03
57168,412,3598,2.0,2001-06-21


In [26]:
df[df.movieId == 65665]

,userId,movieId,rating,date
42243,303,65665,4.0,2011-03-06


In [27]:
# Replace movieId of duplicate Hamlet to the original one 
df['movieId'] = df['movieId'].replace(65665,3598)

In [28]:
# Repetition of War of the Worlds(2005)
df_movie[df_movie.title.str.contains('War of the Worlds')]

,movieId,title,genres,year
2133,2662,"War of the Worlds, The (1953)",Action|Drama|Sci-Fi,1953
6172,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller,2005
7127,64997,War of the Worlds (2005),Action|Sci-Fi,2005


In [29]:
# Remove duplicate from movie dataframe
df_movie.drop(7127,inplace=True)

In [30]:
df[df.movieId == 34048].tail(5)

,userId,movieId,rating,date
92737,615,34048,2.5,2016-05-19
94545,624,34048,3.0,2006-11-25
95653,627,34048,1.0,2008-01-26
98049,654,34048,4.0,2006-04-18
99040,664,34048,3.0,2014-03-04


In [31]:
df[df.movieId == 64997]

,userId,movieId,rating,date
99131,664,64997,2.5,2012-07-31


In [32]:
# Keep latest rating of user 664
df.drop(99131,inplace=True)

### Merge Dataframes

In [33]:
df = pd.merge(df,df_movie, on='movieId')

In [34]:
df.head()

,userId,movieId,rating,date,title,genres,year
0,1,31,2.5,2009-12-14,Dangerous Minds (1995),Drama,1995
1,7,31,3.0,1996-12-29,Dangerous Minds (1995),Drama,1995
2,31,31,4.0,2010-05-11,Dangerous Minds (1995),Drama,1995
3,32,31,4.0,1996-06-15,Dangerous Minds (1995),Drama,1995
4,36,31,3.0,1996-11-03,Dangerous Minds (1995),Drama,1995


In [35]:
# There is one less rating due to the deletion of War of the Worlds from 664
len(df)

100003

In [36]:
df.movieId = df.movieId.astype(str)

In [37]:
# Add m to distinguish movieId from userId
df['movieId'] = df['movieId'].apply(lambda x: 'm' + str(x))

In [38]:
df.to_csv('dataframe.csv', date_format='%Y%m%d')

In [39]:
df_movie['movieId'] = df_movie['movieId'].apply(lambda x: 'm' + str(x))

In [40]:
df_movie.to_csv('dataframe_movie.csv',date_format = '%Y%m%d')

### Users Dataframe

In [41]:
df_user = pd.read_csv('ml-latest-small/tags.csv')

In [42]:
df_user.head(5)

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [43]:
df_user = df_user.drop('timestamp',axis=1)
df_user['movieId'] = df_user['movieId'].apply(lambda x: 'm' + str(x))

In [44]:
# merge dataframes
df2 = pd.merge(df,df_user, on=['userId','movieId'])
df2.head(5)

,userId,movieId,rating,date,title,genres,year,tag
0,212,m1061,4.0,2009-09-26,Sleepers (1996),Thriller,1996,emotional
1,212,m1061,4.0,2009-09-26,Sleepers (1996),Thriller,1996,revenge
2,212,m1061,4.0,2009-09-26,Sleepers (1996),Thriller,1996,true story
3,547,m1172,5.0,2013-07-06,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989,holes80s
4,77,m2968,4.0,2006-11-08,Time Bandits (1981),Adventure|Comedy|Fantasy|Sci-Fi,1981,Gilliam


In [45]:
# Bad comments agree with the rating assigned
df2[df2.tag.str.contains('bad')].head(5)

,userId,movieId,rating,date,title,genres,year,tag
246,346,m33794,1.0,2006-09-24,Batman Begins (2005),Action|Crime|IMAX,2005,bad camerawork
390,212,m27904,3.0,2009-09-26,"Scanner Darkly, A (2006)",Animation|Drama|Mystery|Sci-Fi|Thriller,2006,interesting concept - bad execution
521,212,m68319,3.5,2009-09-26,X-Men Origins: Wolverine (2009),Action|Sci-Fi|Thriller,2009,bad plot
532,212,m69526,2.5,2009-09-26,Transformers: Revenge of the Fallen (2009),Action|Adventure|Sci-Fi|IMAX,2009,bad plot
563,277,m82461,2.0,2016-05-22,Tron: Legacy (2010),Action|Adventure|Sci-Fi|IMAX,2010,bad acting


In [46]:
# The same holds for good critiques
df2[df2.tag.str.contains('good')].head(5)

,userId,movieId,rating,date,title,genres,year,tag
29,431,m661,3.0,2006-02-20,James and the Giant Peach (1996),Adventure|Animation|Children|Fantasy|Musical,1996,creepy good
93,531,m1028,5.0,2009-04-22,Mary Poppins (1964),Children|Comedy|Fantasy|Musical,1964,villain nonexistent or not needed for good story
123,364,m1265,5.0,2015-10-11,Groundhog Day (1993),Comedy|Fantasy|Romance,1993,feel-good
389,212,m27904,3.0,2009-09-26,"Scanner Darkly, A (2006)",Animation|Drama|Mystery|Sci-Fi|Thriller,2006,good animation
529,446,m68954,4.0,2015-07-11,Up (2009),Adventure|Animation|Children|Drama,2009,feel good


### Users and Movies Attributes

In [47]:
genres = sorted(['Adventure','Animation','Children','Drama','Documentary','Horror','Musical','Thriller','Action',
              'Sci-Fi','War','Romance','Comedy','Crime','Fantasy','Western','Mystery','IMAX','Film-Noir'])

In [48]:
# Split string of genres into list of genres for each entry
df['genres'] = df['genres'].apply(lambda x: x.split('|'))

In [49]:
# merge the list of genres for all titles watched by each user
g = df.groupby('userId')[['genres']].agg(sum)

In [50]:
# Return normalized values of interest in genres for each user
def normalize(d, target=1.0):
    
    raw = sum(d.values())
    
    factor = target/raw
     
    return {key: round(value*factor,2) for key,value in d.items()}

In [51]:
# Return three most favorite genres for each user
def keys_max(d):
    
    v=list(d.values())
    k=list(d.keys())
    
    k_1,k_2,k_3 = sorted(d,key = lambda key: d[key],reverse = True)[0:3]   
    
    return k_1,k_2,k_3

In [52]:
genres_norm = g['genres'].apply(lambda x: dict(collections.Counter(x))).apply(normalize).to_dict()

In [53]:
genre_df = pd.DataFrame.from_dict(genres_norm)
genre_df = genre_df.transpose()
genre_df.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,0.09,0.17,0.06,0.04,0.09,0.04,NaN,0.13,0.07,NaN,0.04,NaN,0.02,NaN,0.02,0.09,0.11,0.02,0.02
2,0.09,0.06,0.03,0.04,0.16,0.04,NaN,0.19,0.05,NaN,0.02,0.01,0.02,0.02,0.11,0.02,0.10,0.02,0.01
3,0.11,0.07,0.03,0.03,0.13,0.07,0.02,0.17,0.03,NaN,0.03,0.03,0.01,0.01,0.07,0.05,0.08,0.04,0.01
4,0.11,0.11,0.04,0.07,0.15,0.06,0.00,0.10,0.06,0.0,0.03,0.00,0.04,0.02,0.05,0.07,0.08,0.01,0.00
5,0.05,0.08,0.04,0.05,0.21,0.04,0.01,0.15,0.05,NaN,0.02,0.02,0.04,0.02,0.14,0.03,0.05,0.02,NaN


In [54]:
genres_rank = g['genres'].apply(lambda x: dict(collections.Counter(x))).apply(keys_max).to_dict()

In [55]:
genre_df2 = pd.DataFrame.from_dict(genres_rank)
genre_df2 = genre_df2.transpose()
genre_df2.columns = ['genre1','genre2','genre3']
genre_df2.head()

,genre1,genre2,genre3
1,Adventure,Drama,Thriller
2,Drama,Comedy,Romance
3,Drama,Comedy,Action
4,Comedy,Adventure,Action
5,Comedy,Drama,Romance


## Store data in pkl files 

Year of release movies:

In [56]:
year_dict = df.set_index('movieId')['year'].to_dict()

In [57]:
f = open("year_dict.pkl","wb")
pickle.dump(year_dict,f)
f.close()

Genres of movies:

In [58]:
genre_mov_dict = df.set_index('movieId')['genres'].to_dict()

In [59]:
f = open("genre_mov_dict.pkl","wb")
pickle.dump(genre_mov_dict,f)
f.close()

Title of movies:

In [60]:
title_dict = df.set_index('movieId')['title'].to_dict()

f = open("title_dict.pkl","wb")
pickle.dump(title_dict,f)
f.close()

Normalized genre preference for users

In [61]:
f = open("genre_norm_user.pkl","wb")
pickle.dump(genres_norm,f)
f.close()

Ranked genres for users

In [62]:
f = open("genre_rank_user.pkl","wb")
pickle.dump(genres_rank,f)
f.close()

In [67]:
genre_1_us = {u:k[0] for u,k in genres_rank.items()}

In [68]:
genre_2_us = {u:k[1] for u,k in genres_rank.items()}

In [69]:
f = open("genre_1_us.pkl","wb")
pickle.dump(genre_1_us,f)
f.close()

In [70]:
f = open("genre_2_us.pkl","wb")
pickle.dump(genre_2_us,f)
f.close()

To acknowledge use of the dataset in publications, I cite the following paper:

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872